In [ ]:
from google.colab import files

# Choose the file(s) from your local machine to upload
uploaded = files.upload()

Saving clf-data.zip to clf-data.zip


In [ ]:
!unzip clf-data.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: clf-data/empty/00000300_00000303.jpg  
  inflating: clf-data/empty/00000300_00000319.jpg  
  inflating: clf-data/empty/00000300_00000335.jpg  
  inflating: clf-data/empty/00000300_00000341.jpg  
  inflating: clf-data/empty/00000300_00000344.jpg  
  inflating: clf-data/empty/00000300_00000351.jpg  
  inflating: clf-data/empty/00000300_00000360.jpg  
  inflating: clf-data/empty/00000300_00000377.jpg  
  inflating: clf-data/empty/00000300_00000385.jpg  
  inflating: clf-data/empty/00000300_00000389.jpg  
  inflating: clf-data/empty/00000310_00000004.jpg  
  inflating: clf-data/empty/00000310_00000012.jpg  
  inflating: clf-data/empty/00000310_00000026.jpg  
  inflating: clf-data/empty/00000310_00000029.jpg  
  inflating: clf-data/empty/00000310_00000032.jpg  
  inflating: clf-data/empty/00000310_00000061.jpg  
  inflating: clf-data/empty/00000310_00000072.jpg  
  inflating: clf-data/empty/00000310_0

In [ ]:
# process data
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle

dir_path = 'clf-data'
categories = ['empty', 'not_empty']

data = []
labels = []

for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(dir_path, category)):
        img_path = os.path.join(dir_path, category, file)
        img = imread(img_path)
        img = resize(img, (15,15))
        data.append(img.flatten())
        labels.append(category_idx)

data = np.asarray(data)
labels = np.asarray(labels)

In [ ]:
# split data
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)
print(len(x_train))
print(len(x_test))

4872
1218


In [ ]:
# train classifier
classifier = SVC()
parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C':[1, 10, 100, 1000]}]
grid_search = GridSearchCV(classifier, parameters)

grid_search.fit(x_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001]}])

In [ ]:
# test classifier
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(x_test)
score = accuracy_score(y_pred, y_test)

print(f"{str(score*100)}% of images been classified correctly")

100.0% of images been classified correctly


In [ ]:
# save the model
pickle.dump(grid_search, open('spot_model.p', 'wb'))